In [27]:
from datetime import datetime
import json
import math
import pandas as pd
import requests
import tqdm

In [28]:
url = "https://histdatafeed.vps.com.vn/industry"
response = requests.get(url)
data = response.json()
industries = data.get('data', [])
industries_data_frame = pd.DataFrame(industries)
industries_data_frame.rename(columns={"IcbName": "name", "IcbNameEn": "english_name"})
industries_data_frame.to_csv("./csv/industries.csv", index=False)

In [29]:
url = "https://bgapidatafeed.vps.com.vn/listvn30"
response = requests.get(url)
vn30 = response.json()
vn30_file = open("./vn30.json", "w")
vn30_json = json.dumps(vn30, indent=2)
vn30_file.write(vn30_json)
vn30_file.close()

In [30]:
url = "https://bgapidatafeed.vps.com.vn/getlistallstock"
response = requests.get(url)
companies : list[dict] = response.json()
companies_data_frame = pd.DataFrame(companies)
companies_data_frame.to_csv("./csv/companies.csv", index=False)

In [31]:
timestamp = math.floor(datetime.now().timestamp())
timestamp

1700146145

In [32]:
def get_history(symbol : str):
    now = math.floor(datetime.now().timestamp())
    url = f"https://histdatafeed.vps.com.vn/tradingview/history?symbol={symbol}&resolution=1D&from=0&to={now}"
    response = requests.get(url)
    history_data : dict = response.json()
    symbol = history_data.get("symbol", "")
    t : list[int] = history_data.get("t", [])
    o : list[float] = history_data.get("o", [])
    h : list[float] = history_data.get("h", [])
    l : list[float] = history_data.get("l", [])
    c : list[float] = history_data.get("c", [])
    v : list[int] = history_data.get("v", [])
    history : list[dict] = []
    if len(t) > 0:
        for index, timestamp in enumerate(t):
            date = datetime.fromtimestamp(timestamp).date()
            history.append({
                "date": date,
                "timestamp": timestamp,
                "open": o[index],
                "high": h[index],
                "low": l[index],
                "close": c[index],
                "volume": v[index]
            })
        history_data_frame = pd.DataFrame(history)
        history_data_frame.to_csv(f"./csv/history/{symbol}.csv", index=False)
    

In [33]:
for symbol in vn30:
    get_history(symbol)

In [34]:
list_of_stock = list(filter(lambda c: len(c.get("stock_code")) == 3, companies))
len(list_of_stock)

1615

In [35]:
list_of_stock.reverse()
for stock in tqdm.notebook.tqdm(list_of_stock):
    symbol : str = stock.get("stock_code", "")
    get_history(symbol)

  0%|          | 0/1615 [00:00<?, ?it/s]